## <center> <font color ="purple"> Download and Clean Dataset<font>

<strong>The dataset is about the compressive strength of different samples of concrete based on the volumes of the different ingredients that were used to make them. Ingredients include:</strong>

<strong>1. Cement</strong>

<strong>2. Blast Furnace Slag</strong>

<strong>3. Fly Ash</strong>

<strong>4. Water</strong>

<strong>5. Superplasticizer</strong>

<strong>6. Coarse Aggregate</strong>

<strong>7. Fine Aggregate</strong>

In [2]:
import pandas as pd
import numpy as np

In [3]:
concrete_data = pd.read_csv('concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


#### Let's check how many data points we have.

In [4]:
concrete_data.shape

(1030, 9)

we have so many data to train so we should be careful about data overfitting

## <font color ="red">Data Preprocessing  <font>

In [5]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [6]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

It looks like there is no misssing values

## <font color ="red"> Split data into predictors and target<font>

In [20]:
concrete_data_columns = concrete_data.columns
predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

In [21]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [22]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

In [44]:
#Finally, the last step is to normalize the data by substracting the mean and dividing by the standard deviation.
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [23]:
n_cols = predictors.shape[1] # number of predictors
n_cols

8

## <font color ="red"> Import Keras <font>

In [25]:
import keras
from keras.models import Sequential
from keras.layers import Dense

In [46]:
#Define our model
#our model has one hidden layer with 10 neurons and a ReLU activation function. It uses the adam optimizer and the mean squared error as the loss function.
model = Sequential()
model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
model.add(Dense(10, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1))

# compile model
model.compile(optimizer='adam', loss='mean_squared_error')

Let's import scikit-learn in order to randomly split the data into a training and test sets

In [47]:
from sklearn.model_selection import train_test_split

Splitting the data into a training and test sets by holding 30% of the data for testing

In [48]:
X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3, random_state=42)

## <font color ="red"> Train and Test the Network<font>

In [49]:
# fit the model .we will train the model for 50 epochs.
epochs = 50
model.fit(X_train, y_train, epochs=epochs, verbose=1)

Epoch 1/50
23/23 [==============================] - 0s 771us/step - loss: 41045.1367
Epoch 2/50
23/23 [==============================] - 0s 634us/step - loss: 3621.0942
Epoch 3/50
23/23 [==============================] - 0s 684us/step - loss: 482.0975
Epoch 4/50
23/23 [==============================] - 0s 661us/step - loss: 392.6656
Epoch 5/50
23/23 [==============================] - 0s 680us/step - loss: 323.6333
Epoch 6/50
23/23 [==============================] - 0s 679us/step - loss: 296.2023
Epoch 7/50
23/23 [==============================] - 0s 634us/step - loss: 276.0023
Epoch 8/50
23/23 [==============================] - 0s 699us/step - loss: 262.5830
Epoch 9/50
23/23 [==============================] - 0s 681us/step - loss: 252.6716
Epoch 10/50
23/23 [==============================] - 0s 770us/step - loss: 245.5965
Epoch 11/50
23/23 [==============================] - 0s 680us/step - loss: 239.1468
Epoch 12/50
23/23 [==============================] - 0s 741us/step - loss: 234.595

In [50]:
#we need to evaluate the model on the test data.
loss_val = model.evaluate(X_test, y_test)
y_pred = model.predict(X_test)
loss_val

10/10 [==============================] - 0s 664us/step


192.3894500732422

In [51]:
from sklearn.metrics import mean_squared_error

In [52]:
#we need to compute the mean squared error between the predicted concrete strength and the actual concrete strength.
mean_square_error = mean_squared_error(y_test, y_pred)
mean = np.mean(mean_square_error)
standard_deviation = np.std(mean_square_error)
print(mean, standard_deviation)

192.3894412241612 0.0


In [53]:
#Create a list of 50 mean squared errors and report mean and the standard deviation of the mean squared errors.
total_mean_squared_errors = 50
epochs = 50
mean_squared_errors = []
for i in range(0, total_mean_squared_errors):
    X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3, random_state=i)
    model.fit(X_train, y_train, epochs=epochs, verbose=0)
    MSE = model.evaluate(X_test, y_test, verbose=0)
    print("MSE "+str(i+1)+": "+str(MSE))
    y_pred = model.predict(X_test)
    mean_square_error = mean_squared_error(y_test, y_pred)
    mean_squared_errors.append(mean_square_error)

mean_squared_errors = np.array(mean_squared_errors)
mean = np.mean(mean_squared_errors)
standard_deviation = np.std(mean_squared_errors)

print('\n')
print("Below is the mean and standard deviation of " +str(total_mean_squared_errors) + " mean squared errors without normalized data. Total number of epochs for each training is: " +str(epochs) + "\n")
print("Mean: "+str(mean))
print("Standard Deviation: "+str(standard_deviation))

MSE 1: 155.63064575195312
10/10 [==============================] - 0s 775us/step
MSE 2: 124.51069641113281
10/10 [==============================] - 0s 664us/step
MSE 3: 86.5258560180664
10/10 [==============================] - 0s 670us/step
MSE 4: 57.39324951171875
10/10 [==============================] - 0s 665us/step
MSE 5: 50.19718551635742
10/10 [==============================] - 0s 664us/step
MSE 6: 49.73033142089844
10/10 [==============================] - 0s 1ms/step
MSE 7: 55.326507568359375
10/10 [==============================] - 0s 888us/step
MSE 8: 44.255393981933594
10/10 [==============================] - 0s 1ms/step
MSE 9: 45.71715545654297
10/10 [==============================] - 0s 665us/step
MSE 10: 50.430335998535156
10/10 [==============================] - 0s 553us/step
MSE 11: 41.891117095947266
10/10 [==============================] - 0s 665us/step
MSE 12: 47.139522552490234
10/10 [==============================] - 0s 1ms/step
MSE 13: 47.650840759277344
10/10 [===